##  4. 통계기반 데이터 분석
1. 회귀
2. 로지스틱 회귀
3. 회귀 알고리즘을 활용한 추정과 예측

---

### 1. 회귀
- 하나의 종속변수(예측하고자 하는 값)와 독립변수(예측자) 사이의 관계를 명시하는 것
- 예)
  - 경제, 사회학, 심리학, 물리학, 생태학 같은 분야에서 측정된 특성으로, 모집단과 개별이 어떻게 다른지 평가한다.
  - 임상 약품 실험, 엔지니어링 안전검사, 시장 연구 같은 사건과 결과 간에 인과관계를 수량화한다.
  - 보험 청구, 자연 재해 손해, 선거 결과, 범죄율 예측 같은 주어진 기준으로 미래의 행위를 예측하기 위해 사용되는 패턴을 식별한다.

- 잔차 : 예측값과 실제값의 차이

- 정규 최소 제곱 추정
  - 정규최소제곱(OLS, Ordinary Least Squares)로 알려진 추정 기법
  - MSE(Mean Squares Error)의 지표

- 다중 선형 회귀


 ---
 Sckit-learn 중 적절한 회귀 모델의 선택
 - LinearRegression
  - 데이터 셋이 많고 속성의 개수가 적은 선형모델에 적합
  - 정규방정식(Normal Equation을 사용함)
 - SGDRegressor
  - 데이터 셋이 많고, 속성의 개수가 많은 모델에 적합

#### 선형회귀 구현

##### 데이터 불러오기

In [1]:
import pandas as pd
import seaborn as sns

dia = sns.load_dataset('diamonds')

##### 독립변수, 종속변수 나누기

In [3]:
X = dia[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = dia['price']
print(X.shape, y.shape)

(53940, 6) (53940,)


##### 학습데이터, 테스트데이터 나누기

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [6]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(43152, 6) (10788, 6) (43152,) (10788,)


##### 모델 학습

In [7]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

##### 모델 예측

In [11]:
pred = lr.predict(X_test)

##### 모델 성능확인

In [12]:
lr.score(X_test, y_test)

0.8702801037706538

In [13]:
from sklearn.metrics import r2_score, mean_squared_error
print('r2_score : ', r2_score(y_test, pred))
print('mean_squared_error : ', mean_squared_error(y_test, pred))
print('root_mean_squared_error : ', mean_squared_error(y_test, pred, squared=False))

r2_score :  0.8702801037706538
mean_squared_error :  2003751.981045021
root_mean_squared_error :  1415.5394664385099


In [20]:
print('coef_ : ', lr.coef_)
print('intercept_ : ', lr.intercept_)

coef_ :  [10505.14109349  -200.45512942  -101.25909375 -1242.13612974
    66.45437265    40.28657298]
intercept_ :  20342.88110610444


In [21]:
pd.DataFrame(lr.coef_, columns=['coef'], index=X.columns)

,coef
carat,10505.141093
depth,-200.455129
table,-101.259094
x,-1242.136130
y,66.454373
z,40.286573


##### 여러 속성에 대한 선형 회귀의 적용
- PolynomialFeatures 를 이용해 전처리

In [22]:
 from sklearn.preprocessing import PolynomialFeatures
 poly_features = PolynomialFeatures(degree=2, include_bias=False)
 X_poly = poly_features.fit_transform(X)

##### 규제 항이 추가 된 회귀모델
- NORM???
- 규제방법에는
  - 릿지 : 계수의 제곱의 합 / 2
  - 라쏘 : 계수의 절대 값의 합
  - 엘라스틱
    - 릿지, 라쏘의 규제항을 혼합한 선형회귀
    - r이 0이면 릿지, r이 1이면 라쏘가 됨

- alpha 파라미터로 규제항의 가중치?? 를 조절

- 릿지, 라쏘, 엘라스틱 - 코드,그래프 구현

- 목적함수 = 손실항(MSE) + 규제항

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [30]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=10.0)
ridge.fit(X_train_scaled, y_train)
ridge.score(X_test_scaled, y_test)

0.8701985055999106

In [31]:
ridge = Ridge(alpha=0.1)
ridge.fit(X_train_scaled, y_train)
ridge.score(X_test_scaled, y_test)

0.8702793053626318

In [32]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=10.0)
lasso.fit(X_train_scaled, y_train)
lasso.score(X_test_scaled, y_test)

0.8686130940359574

In [33]:
from sklearn.linear_model import ElasticNet
elastic = ElasticNet(alpha=10.0, l1_ratio=0.5)
elastic.fit(X_train_scaled, y_train)
elastic.score(X_test_scaled, y_test)

0.5537993861932904

#### 로지스틱 회귀
- 종속변수의 값을 0또는 1 (이진변수 로) 표현할 수 있는 경우 사용
- 수치형 데이터만 사용가능
- 훈련하는데 대부분의 시간이 소요되며, 이 단계에서 데이터를 분류하기 위해 최적의 계수를 찾음
  - 훈련단계를 마치고 나면 분류는 빠르고 쉽게 진행 됨.
- 장점
  - 계산비용이 적음
  - 구현이 쉬움
  - 결과해석을 위한 지식표현이 쉬움
- 단점
  - 언더피팅(Underfitting) 경향이 있어, 정확도가 낮게 나올 수 있음

##### 붓꽃 데이터로 이진분류

In [34]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename',
 'data_module']

In [36]:
import numpy as np

X = iris['data'][:, 3:]
y = (iris['target'] == 2).astype(np.int32)
X.shape, y.shape

((150, 1), (150,))

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

import pandas as pd
print(pd.Series(y_train).value_counts())
print(pd.Series(y_test).value_counts())

0    80
1    40
Name: count, dtype: int64
0    20
1    10
Name: count, dtype: int64


##### 로지스틱 회귀

In [48]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [49]:
log_reg.score(X_test, y_test)

0.9666666666666667